In [1]:
using StatsBase
using GLM
using DataFrames
using Plots

INFO: Precompiling module GLM.


In [2]:
plotlyjs()

Plots.PlotlyJSBackend()

In [3]:
# Get the relevant data

prob = readdlm("dat/clf_groups_M20_M10.3.prob")
P_sat = prob[:, 6]
cen_ids = P_sat .< 0.01

corr = readdlm("dat/clf_groups_M20_M10.3.galdata_corr")
Dn4000 = corr[:, 6]
quench_ids = Dn4000 .> 1.6

samp_ids = cen_ids .* quench_ids

density = readdlm("dat/density_r10.M20_M10.3")
rands = readdlm("dat/drandom_r10.M20_M10.3")
ρ_corr = 1.25 * (density[:, 1] ./ rands[:, 1])
nonzero = ρ_corr .> 0.0

samp_ids = samp_ids .* nonzero

75293-element BitArray{1}:
  true
  true
 false
  true
  true
  true
 false
  true
 false
  true
  true
 false
 false
     ⋮
  true
  true
  true
 false
  true
  true
  true
 false
 false
 false
 false
 false

In [61]:
histogram(log10.(corr[:, 9]))

In [62]:
corr2 = readdlm("dat/clf_groups_M20_M10.2.galdata_corr")
histogram(log10.(corr2[:, 9]))

In [63]:
corr3 = readdlm("dat/clf_groups_M19_M9.8.galdata_corr")
histogram(log10.(corr3[:, 9]))

In [67]:
histogram(prob[:, 5])

In [68]:
prob2 = readdlm("dat/clf_groups_M19_M9.8.prob")
histogram(prob2[:, 5])

In [44]:
histogram(ρ_corr)

In [55]:
ρ_bar = mean(ρ_corr[samp_ids])

2.647634430535747

In [32]:
histogram(Dn4000[cen_ids], bins=collect(0:.02:2.3), xlabel="Dn4000")

In [46]:
δ = ρ_corr ./ ρ_bar
mean_δ = mean(δ)


1.0

In [47]:
histogram(δ)

In [21]:
logδ = log10.(ρ_corr[samp_ids])
Dn4k = Dn4000[samp_ids]

42560-element Array{Any,1}:
 1.72142
 1.93181
 1.99379
 1.81657
 1.81796
 1.67551
 1.85702
 1.86562
 1.73672
 1.6588 
 1.93528
 1.88434
 1.61887
 ⋮      
 1.93522
 2.0044 
 1.65802
 1.78256
 1.89461
 1.84494
 1.98744
 1.97426
 2.02718
 1.85615
 1.97743
 1.86615

In [22]:
count(!isfinite, logδ)

0

In [23]:
df = DataFrame(a=logδ, b=Dn4k)

,a,b
1,0.13236474433292647,1.72142
2,0.32865938358360935,1.93181
3,0.2614594566638992,1.99379
4,0.37124673850215817,1.81657
5,0.8412431028447035,1.81796
6,-1.0048083763155244,1.67551
7,0.5892825898109387,1.85702
8,0.1323647659416388,1.86562
9,0.1859475462518949,1.73672
10,0.1993115339635397,1.6588


In [24]:
# ols = glm(@formula(a ~ b), df, Normal(), IdentityLink())
lm1 = fit(LinearModel, @formula(a ~ b), df)

DataFrames.DataFrameRegressionModel{GLM.LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Formula: a ~ 1 + b

Coefficients:
              Estimate  Std.Error t value Pr(>|t|)
(Intercept)  0.0942749  0.0169918 5.54825    <1e-7
b              0.11587 0.00904844 12.8055   <1e-36


In [25]:
cor(logδ, Dn4k)

0.061954187622677945

In [29]:
subset = sample(1:42560, 10000, replace=false)
scatter(Dn4k[subset], logδ[subset], markersize=.1, xlims = (1.6, 2.8))

In [30]:
f(x) = 0.0942749 + 0.11587 * x
plot!(f, 1.6, 2.8, xlabel="Dn4000", ylabel="log(1 + δ)")
annotate!([(2.2,0.1,text("log(1 + δ) = 0.094 + 0.12 Dn4000",8,:red,:left))])
annotate!([(2.2,-0.1,text("r = 0.062",8,:red,:left))])

In [39]:
Dnbins = collect(1.6:0.05:2.1)

11-element Array{Float64,1}:
 1.6 
 1.65
 1.7 
 1.75
 1.8 
 1.85
 1.9 
 1.95
 2.0 
 2.05
 2.1 

In [40]:
mD = [mean(Dn4k[Dnbins[i] .<= Dn4k .< Dnbins[i + 1]]) for i in 1:10]
mlδ = [mean(logδ[Dnbins[i] .<= Dn4k .< Dnbins[i + 1]]) for i in 1:10]

10-element Array{Float64,1}:
 0.255044
 0.264177
 0.278705
 0.278923
 0.29838 
 0.301318
 0.321453
 0.347688
 0.364499
 0.370056

In [53]:
plot(mD, mlδ, xlabel="Dn4000", ylabel="log10(1 + δ)", yticks=collect(0.26:0.02:0.36))

In [36]:
histogram(logδ)

In [42]:
δbins = collect(-1:0.2:1.2)
mD2 = [mean(Dn4k[δbins[i] .<= logδ .< δbins[i + 1]]) for i in 1:11]
mlδ2 = [mean(logδ[δbins[i] .<= logδ .< δbins[i + 1]]) for i in 1:11]

11-element Array{Float64,1}:
 -0.876584 
 -0.674242 
 -0.480004 
 -0.28312  
 -0.0862342
  0.107629 
  0.303593 
  0.493742 
  0.684412 
  0.882006 
  1.06484  

In [54]:
plot(mlδ2, mD2, xlabel="log10(1 + δ)", ylabel="Dn4000")

In [9]:
df2 = DataFrame(d=logδ, c=Dn4k)
lm2 = fit(LinearModel, @formula(c ~ d), df2)


LoadError: [91mMethodError: no method matching getindex(::DataFrames.DataFrame, ::DataArrays.DataArray{Any,1}, ::Colon)[39m

In [26]:
f(x) = 0.217076 + 0.2668 * x
cor(logδ, Dn4k)

0.06195418762267758

In [39]:
subset = sample(1:42560, 10000, replace=false)
scatter(Dn4k[subset], logδ[subset], markersize=.1, xlims = (1.6, 2.8))

In [40]:
plot!(f, 1.6, 2.8, xlabel="log(δ)", ylabel="Dn4000")
annotate!([(2.2,0.1,text("Dn4000 = 0.217 + 0.267 log(δ)",8,:red,:left))])
annotate!([(2.2,-0.1,text("r = 0.062",8,:red,:left))])

In [41]:
lm2 = fit(LinearModel, @formula(b ~ a), df)

LoadError: [91mMethodError: no method matching getindex(::DataFrames.DataFrame, ::DataArrays.DataArray{Any,1}, ::Colon)[39m

In [10]:
scatter(logδ[subset], Dn4k[subset], markersize=.1, ylims = (1.6, 2.8))